In [ ]:
from urllib.parse import urlencode, quote_plus
from urllib.request import urlopen, Request
import pandas as pd
import json, xmltodict
import pandas_profiling

service_key = 'VtS0z8ILedUMwjLe4kgmh6T8MWir/F9BsncSDqxFPsMsJaeTDkw3fDhNisjV5GdZerEeaEhxwDYbDhRG2SDz1g=='

In [ ]:
def get_gongdong_price(pnu, stdrYear, numOfRows, pageNo):    
    queryParams = '?' + urlencode({ quote_plus('ServiceKey') : service_key, quote_plus('pnu') : pnu, quote_plus('stdrYear') : stdrYear, quote_plus('format') : 'json', quote_plus('numOfRows') : numOfRows, quote_plus('pageNo') : pageNo })
    
    url = 'http://apis.data.go.kr/1611000/nsdi/ApartHousingPriceService/attr/getApartHousingPriceAttr'
    request = Request(url + queryParams)
    request.get_method = lambda: 'GET'
    response_body = urlopen(request).read().decode('utf-8')
    data = json.loads(response_body)
    
    return data

In [ ]:
def get_price_yeonrib(LAWD_CD, DEAL_YMD):
    queryParams = '?' + urlencode({ quote_plus('ServiceKey') : service_key, quote_plus('LAWD_CD') : LAWD_CD, quote_plus('DEAL_YMD') : DEAL_YMD })

    url_yeonrib = 'http://openapi.molit.go.kr:8081/OpenAPI_ToolInstallPackage/service/rest/RTMSOBJSvc/getRTMSDataSvcRHTrade'
    request = Request(url_yeonrib + queryParams)
    request.get_method = lambda: 'GET'
    response_body = urlopen(request).read().decode('utf-8')
    data = json.loads(json.dumps(xmltodict.parse(response_body)))
    
    return data

In [ ]:
code_df = pd.read_csv('/Users/jungwookim/dev/RealEstateInSeoul/beob_code.csv', encoding = "utf-8", index_col='법정동코드')
code_dic = code_df.loc[:,['법정동명']].to_dict()['법정동명']

seoul_code_list = list()
for code in code_dic:
    if code < 2000000000:
        seoul_code_10[code] = code_dic[code]
        
for key in seoul_code_list:
    seoul_code_list.append((str(key)[:5], str(key)[5:], seoul_code_10[key]))

In [ ]:
def get_address(code_list):
    address_list = list()
    for item in code_list:
        # ~구라고 되어있는 곳까지 짜르고
        if item[2].find('구') == -1:
            continue
        else:
            split_point = item[2].find('구')
            tp = (item[0], item[2][:split_point+1], item[1], item[2][split_point+1:].replace(" ", ""))
            address_list.append(tp)
    return address_list

In [ ]:
# 예외 처리 방법
def get_jibunCode(jibun):
    if '-' in jibun:
        split_point = jibun.find('-')
        if len(jibun[:split_point]) == 1:
            if len(jibun[split_point+1:]) == 1:
                return '000'+jibun[:split_point]+'000'+jibun[split_point+1:]
            elif len(jibun[split_point+1:]) == 2:
                return '000'+jibun[:split_point]+'00'+jibun[split_point+1:]
            elif len(jibun[split_point+1:]) == 3:
                return '000'+jibun[:split_point]+'0'+jibun[split_point+1:]
            elif len(jibun[split_point+1:]) == 4:
                return '000'+jibun[:split_point]+jibun[split_point+1:]
        elif len(jibun[:split_point]) == 2:
            if len(jibun[split_point+1:]) == 1:
                return '00'+jibun[:split_point]+'000'+jibun[split_point+1:]
            elif len(jibun[split_point+1:]) == 2:
                return '00'+jibun[:split_point]+'00'+jibun[split_point+1:]
            elif len(jibun[split_point+1:]) == 3:
                return '00'+jibun[:split_point]+'0'+jibun[split_point+1:]
            elif len(jibun[split_point+1:]) == 4:
                return '00'+jibun[:split_point]+jibun[split_point+1:]
        elif len(jibun[:split_point]) == 3:
            if len(jibun[split_point+1:]) == 1:
                return '0'+jibun[:split_point]+'000'+jibun[split_point+1:]
            elif len(jibun[split_point+1:]) == 2:
                return '0'+jibun[:split_point]+'00'+jibun[split_point+1:]
            elif len(jibun[split_point+1:]) == 3:
                return '0'+jibun[:split_point]+'0'+jibun[split_point+1:]
            elif len(jibun[split_point+1:]) == 4:
                return '0'+jibun[:split_point]+jibun[split_point+1:]
        elif len(jibun[:split_point]) == 1:
            if len(jibun[split_point+1:]) == 4:
                return jibun[:split_point]+'000'+jibun[split_point+1:]
            elif len(jibun[split_point+1:]) == 2:
                return jibun[:split_point]+'00'+jibun[split_point+1:]
            elif len(jibun[split_point+1:]) == 3:
                return jibun[:split_point]+'0'+jibun[split_point+1:]
            elif len(jibun[split_point+1:]) == 4:
                return jibun[:split_point]+jibun[split_point+1:]
    else:
        if len(jibun) == 1:
            return '000'+jibun+'0000'
        elif len(jibun) == 2:
            return '00'+jibun+'0000'
        elif len(jibun) == 3:
            return '0'+jibun+'0000'
        elif len(jibun) == 4:
            return jibun+'0000'

In [ ]:
data2 = get_price_yeonrib('11110', '201712')
df2 = pd.DataFrame(data2['response']['body']['items']['item'])
df2.head()

In [ ]:
# dongCode 추가

address_list = get_address(seoul_code_list)
dong_list = list()
for a, row in df2.iterrows():
    for item in address_list:
        if row['지역코드'] == item[0]:
            if row['법정동'] == item[3]:
                dong_list.append(item[2])

df2['dongCode'] = dong_list

In [ ]:
# jibunCode 추가
df2['jibunCode'] = df2['지번'].map(get_jibunCode)

In [ ]:
# 최종 pnu 생성
df2['pnu'] = df2['지역코드'] + df2['dongCode'] + '1' + df2['jibunCode']